# pyGemPick Tutorial 3: Outputing Detected Gold Particle Centers

## How to Output Gold Particle Centers To Use In Future Spatial-Statistical Analysis of Gold Particle Cross Correlation

In this tutorial we'll be using the **bin2df( )** function found in the pygempick.spatialstats package to record the x,y centers of each detected keypoint in each image!

```python

def bin2df(images):
    
    i = 0  #image counter
    j = 0 #total particles
    
    #difine filtering paramaters 
    pclap = 25 #HCLAP anchor value
    plog  = 20 #HLOG anchor value 
    
    #make an empty dataframe, which will be filled with x,y centers of keypoints detected
    data = pd.DataFrame()
    
    #Change picking parameters per test set...
    #Will ask you to fill in these parameters - keep it same as parameters used in Tutorial2
    
    minArea = np.int(input('Min Area to Detect: '))
    minCirc = np.float(input('Min Circularity: '))
    minCon = np.float(input('Min Concavity: '))
    minIner = np.float(input('Min Inertial Ratio: '))
    
    for image in images:
        
        orig_img = cv2.imread(image) ##reads specific test file
        output1 = py.hclap_filt(pclap, orig_img, 'no') #High Contrast Laplace Filtering!
        output2 = py.hlog_filt(plog, orig_img, 'no')

        #image, minArea, minCirc, minConv, minIner, minThres
        #use these parameters >> keypoints1 = py.pick(output1, 37, .71, .5 , .5, 0) 
        keypoints1 = py.pick(output1, minArea, minCirc, minCon , minIner, 0) 
        keypoints2 = py.pick(output2, minArea, minCirc, minCon , minIner, 0)
        
        #this function removes duplicated detections
        keypoints1, dup1 = py.key_filt(keypoints1, keypoints2)
        #combine the two lists of keypoints
        keypoints = keypoints1 + keypoints2
        
        #record the subsequent keypoint centers and update the pandas dataframe
        data, k = record_kp(i,keypoints,data)
        
        j += k
            
        i+=1

    return data, j #returns data as df and total particles accounted...
    
```



In [1]:
#import required modules
import glob
import pygempick.spatialstats as spa


In [2]:
images = glob.glob('/home/joseph/Documents/pygempick/samples/compressed/*.jpg') 

In [6]:
N = len(images)
data, gold = spa.bin2df(images)
file = input("Name Your File!")
data.to_csv('/home/joseph/Documents/pygempick/supdocs/{}'.format(file), index=False)
print('Counted {} gold particle in {} images.'.format(gold,N))

Min Area to Detect: 37
Min Circularity: .71
Min Concavity: .5
Min Inertial Ratio: .5
Name Your File!detected.csv
Counted 64 gold particle in 13 images.


In [7]:
# this outputs a csv file which you cn find in the folder above or save to the folder of your choice!
# this dataframe (or file) can be used to calculate the spatial-point distribution of the gold 
# particle positions on these EM mircrographs!